# Prepare Annotation Tables

In [119]:
import pandas as pd
import os
import math
import Bio
from Bio import Seq
import re
import matplotlib.pyplot as plt
import numpy as np
import Bio.Data.CodonTable
table = Bio.Data.CodonTable.ambiguous_dna_by_id[1]

### Get variant Data

In [691]:
variants = pd.read_csv('../4-Variation/variant_list.05.tsv', sep='\t')
variants_no_filter = pd.read_csv('../4-Variation/variant_list.tsv', sep='\t')

In [692]:
synonymous=variants[(variants.EFFECT!='SYNONYMOUS_CODING') & (variants.EFFECT!='.')]

In [693]:
synonymous.GENE.unique()

array(['ORF3a', 'S', 'orf1ab', 'N', 'ORF7a', 'ORF8', 'E', 'ORF10', 'M',
       'ORF6', 'ORF7b'], dtype=object)

In [694]:
synonymous.groupby('POS')['ALT'].value_counts()


POS    ALT
269    T      1
275    T      1
285    T      4
308    A      2
312    A      1
             ..
29516  A      1
29580  A      1
29612  C      1
29625  T      1
29642  T      3
Name: ALT, Length: 1161, dtype: int64

In [695]:
synonymous['NbSample']=0
synonymous['MeanAF']=0
groups=synonymous.groupby('POS')['ALT'].value_counts()
meanAF=synonymous.groupby(['POS','ALT'])['AF'].mean()
for i,row in synonymous.iterrows():
    synonymous.loc[i, 'NbSample']=groups[row['POS']][row['ALT']]
    synonymous.loc[i, 'MeanAF']=meanAF[row['POS']][row['ALT']]

<ipython-input-695-950837d7ae45>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synonymous['NbSample']=0
<ipython-input-695-950837d7ae45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synonymous['MeanAF']=0
/Users/delphinel/anaconda3/envs/jupyter/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [696]:
synonymous

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,IMPACT,FUNCLASS,EFFECT,GENE,CODON,type,NbSample,MeanAF
2,SRR11397714,NC_045512,25563,G,T,119,1.000000,0,"0,0,56,63",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
3,SRR11494669,NC_045512,25563,G,T,160,1.000000,0,"0,0,74,86",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
4,SRR11494527,NC_045512,25563,G,T,137,1.000000,0,"0,0,57,80",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
5,SRR11494644,NC_045512,25563,G,T,203,1.000000,0,"0,0,111,92",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
6,SRR11494701,NC_045512,25563,G,T,149,1.000000,0,"0,0,69,80",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4139,SRR11410542,NC_045512,11751,T,C,139,0.050360,1,"35,97,1,6",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,cTc/cCc,S,1,0.050360
4141,SRR11494532,NC_045512,19159,T,C,159,0.050314,1,"58,93,4,4",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,Tct/Cct,S,1,0.050314
4142,SRR11494664,NC_045512,24131,G,T,179,0.050279,1,"81,89,5,4",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,S,Ggc/Tgc,S,1,0.050279
4143,SRR11494664,NC_045512,23903,G,T,179,0.050279,0,"79,91,4,5",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,S,Gtt/Ttt,S,1,0.050279


### Get Variant Selection Data

In [697]:
selection_data = pd.read_csv('Selective_Analysis_Data/20200418.csv', sep=',')

In [698]:
selection_data

,First Detected,Site,Categories,Site composition/Evolutionary Credibility,Substitutions at <br> Internal Branches <br> [Syn:Non-syn]
0,"[[""20200403"",""20200408"",""20200410"",""20200412"",...",N 13<br/>Genomic <a target = '_blank_' href = ...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span style=\""color:darkgreen\...","[0,3]"
1,"[[""20200325"",""20200330"",""20200403"",""20200404""],9]",ORF3a 57<br/>Genomic <a target = '_blank_' hre...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span title = ''>Q<sub>7872</s...","[0,2]"
2,"[[""20200325"",""20200330"",""20200403"",""20200404"",...",ORF1a 265<br>nsp2 85<br/>Genomic <a target = '...,"{""fel"":true,""meme"":true,""fraction"":0.993106172...","{""composition"":""<span style=\""color:#ff8c00\""t...","[0,4]"
3,"[[""20200408"",""20200410"",""20200412"",""20200414"",...",ORF1a 3353<br>3C-like proteinase 90<br/>Genomi...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span style=\""color:darkgreen\...","[0,3]"
4,"[[""20200325"",""20200330"",""20200403"",""20200404"",...",ORF1a 3606<br>nsp6 37<br/>Genomic <a target = ...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span style=\""color:#c2a90f\""t...","[0,3]"
...,...,...,...,...,...
121,"[[""20200325"",""20200330"",""20200403"",""20200404"",...",ORF1b 1022<br>helicase 100<br/>Genomic <a targ...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span style=\""color:darkgreen\...","[0,1]"
122,"[[""20200415""],9]",ORF1b 1259<br>helicase 337<br/>Genomic <a targ...,"{""fel"":true,""meme"":false,""fraction"":0.15142841...","{""composition"":""<span style=\""color:darkgreen\...","[2,0]"
123,"[[""20200410"",""20200412"",""20200414"",""20200415""],9]",ORF1b 1270<br>helicase 348<br/>Genomic <a targ...,"{""fel"":true,""meme"":false,""fraction"":0.99505757...","{""composition"":""<span style=\""color:darkgreen\...","[0,1]"
124,"[[""20200414"",""20200415""],9]",ORF1b 1443<br>helicase 521<br/>Genomic <a targ...,"{""fel"":true,""meme"":false,""fraction"":0.21441899...","{""composition"":""<span style=\""color:darkgreen\...","[0,0]"


Clean up of data, split collumns and removing html tags

In [699]:
selection_data['syn']=0
selection_data['non_syn']=0
selection_data['REF']=0
selection_data['ALT']=0
selection_data['Genomic_coord_end']=0
selection_data['Genomic_coord_start']=0
selection_data['Gene']=0
selection_data['Main_ref_codon']=0
selection_data['Main_alt_codon']=0
selection_data['Main_ref_residue']=0
selection_data['Main_alt_residue']=0
selection_data['NbSample']=0
selection_data['MeanAF']=0


In [700]:
for i,row in selection_data.iterrows():
    selection_data.loc[i,'syn'] =row['Substitutions at <br> Internal Branches <br> [Syn:Non-syn]'].replace('[','').replace(']','').split(',')[0]
    selection_data.loc[i, 'non_syn']=row['Substitutions at <br> Internal Branches <br> [Syn:Non-syn]'].replace('[','').replace(']','').split(',')[1]
    selection_data.loc[i, 'Genomic_coord_start']=selection_data['Site'][i][:-4].split('>')[-1].split('-')[0]
    selection_data.loc[i, 'Genomic_coord_end']=selection_data['Site'][i][:-4].split('>')[-1].split('-')[1]
    selection_data.loc[i, 'NbSample']=len(row['First Detected'].replace('[[','').replace("],9]",'').split(','))
    locations=selection_data['Site'][i][:-4].split('<br/>')[0].split('<br>')
    selection_data.loc[i, 'Gene']=",".join(locations)
    composition=selection_data['Site composition/Evolutionary Credibility'][i].split(',')[2:]
    if len(composition)>0:
        variation=list((np.unique(np.array([i.replace('"',"") for i in composition if '/' in i]))))
        selection_data.loc[i, 'Main_ref_codon']=list(variation)[0].split('/')[0]
        selection_data.loc[i, 'Main_alt_codon']=','.join([i.split('/')[1] for i in variation])
        selection_data.loc[i, 'Main_ref_residue']=Bio.Seq._translate_str(list(variation)[0].split('/')[0], table)
        selection_data.loc[i, 'Main_alt_residue']=','.join([Bio.Seq._translate_str(i.split('/')[1], table) for i in variation])
    else :
        variation=selection_data['Site composition/Evolutionary Credibility'][i].replace('{"composition":"','').replace("<span title = \'\'>",'').replace('</sub></span>","variants":[]}','').split('</sub></span>')
        dic={}
        variants=''
        for j in variation:
            var=re.sub("\<span.*\\'\>",'',j)
            dic[var.split('<sub>')[0]]=var.split('<sub>')[1] 
            variants=variants+var.split('<sub>')[0]+','
        selection_data.loc[i, 'Main_ref_residue']=variants[0]
        selection_data.loc[i, 'Main_alt_residue']=variants[1:].strip(',')
    
 

Remove unecessary columns

In [701]:
selection_data

,First Detected,Site,Categories,Site composition/Evolutionary Credibility,Substitutions at <br> Internal Branches <br> [Syn:Non-syn],syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,Gene,Main_ref_codon,Main_alt_codon,Main_ref_residue,Main_alt_residue,NbSample,MeanAF
0,"[[""20200403"",""20200408"",""20200410"",""20200412"",...",N 13<br/>Genomic <a target = '_blank_' href = ...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span style=\""color:darkgreen\...","[0,3]",0,3,0,0,28312,28310,N 13,Cgc,Ggc,R,G,6,0
1,"[[""20200325"",""20200330"",""20200403"",""20200404""],9]",ORF3a 57<br/>Genomic <a target = '_blank_' hre...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span title = ''>Q<sub>7872</s...","[0,2]",0,2,0,0,25563,25561,ORF3a 57,caG,"caA,caT",Q,"Q,H",4,0
2,"[[""20200325"",""20200330"",""20200403"",""20200404"",...",ORF1a 265<br>nsp2 85<br/>Genomic <a target = '...,"{""fel"":true,""meme"":true,""fraction"":0.993106172...","{""composition"":""<span style=\""color:#ff8c00\""t...","[0,4]",0,4,0,0,1060,1058,"ORF1a 265,nsp2 85",acC,acT,T,T,9,0
3,"[[""20200408"",""20200410"",""20200412"",""20200414"",...",ORF1a 3353<br>3C-like proteinase 90<br/>Genomi...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span style=\""color:darkgreen\...","[0,3]",0,3,0,0,10324,10322,"ORF1a 3353,3C-like proteinase 90",aaG,aaT,K,N,5,0
4,"[[""20200325"",""20200330"",""20200403"",""20200404"",...",ORF1a 3606<br>nsp6 37<br/>Genomic <a target = ...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span style=\""color:#c2a90f\""t...","[0,3]",0,3,0,0,11083,11081,"ORF1a 3606,nsp6 37",ttG,ttT,L,F,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,"[[""20200325"",""20200330"",""20200403"",""20200404"",...",ORF1b 1022<br>helicase 100<br/>Genomic <a targ...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span style=\""color:darkgreen\...","[0,1]",0,1,0,0,16536,16534,"ORF1b 1022,helicase 100",0,0,S,"I,G",9,0
122,"[[""20200415""],9]",ORF1b 1259<br>helicase 337<br/>Genomic <a targ...,"{""fel"":true,""meme"":false,""fraction"":0.15142841...","{""composition"":""<span style=\""color:darkgreen\...","[2,0]",2,0,0,0,17247,17245,"ORF1b 1259,helicase 337",0,0,R,,1,0
123,"[[""20200410"",""20200412"",""20200414"",""20200415""],9]",ORF1b 1270<br>helicase 348<br/>Genomic <a targ...,"{""fel"":true,""meme"":false,""fraction"":0.99505757...","{""composition"":""<span style=\""color:darkgreen\...","[0,1]",0,1,0,0,17280,17278,"ORF1b 1270,helicase 348",0,0,V,L,4,0
124,"[[""20200414"",""20200415""],9]",ORF1b 1443<br>helicase 521<br/>Genomic <a targ...,"{""fel"":true,""meme"":false,""fraction"":0.21441899...","{""composition"":""<span style=\""color:darkgreen\...","[0,0]",0,0,0,0,17799,17797,"ORF1b 1443,helicase 521",0,0,V,,2,0


In [702]:
smaller=selection_data[['syn','non_syn','REF','ALT','Genomic_coord_end','Genomic_coord_start','Gene','Main_ref_codon','Main_alt_codon','Main_ref_residue','Main_alt_residue','NbSample','MeanAF']]

## Add Feature data  

Main Feature , eg. ORF1a, and secondary Feature, eg. nsp5

In [703]:
smaller['MainFeat']='NA'
smaller['MainResidue']=0
smaller['SecondFeat']='NA'
smaller['SecondResidue']=0
smaller['ThirdFeat']='NA'
smaller['ThirdResidue']=0


smaller['MainResidue']='na'
smaller['MainFeat']='na'
smaller['SecondFeat']='na'
smaller['SecondResidue']='na'
smaller['Second_ref_codon']='na'
smaller['Second_alt_codon']='na'
smaller['Second_ref_residue']='na'
smaller['Second_alt_residue']='na'
smaller['ThirdResidue']='na'
smaller['ThirdFeat']='na'
smaller['Third_ref_codon']='na'
smaller['Third_alt_codon']='na'
smaller['Third_ref_residue']='na'
smaller['Third_alt_residue']='na'

<ipython-input-703-d922c7532572>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller['MainFeat']='NA'
<ipython-input-703-d922c7532572>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller['MainResidue']=0
<ipython-input-703-d922c7532572>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [704]:
for i,row in smaller.iterrows():
    liste_loc=row['Gene'].split(',')
    smaller.loc[i, 'MainFeat']=" ".join(liste_loc[0].split(" ")[:-1])
    smaller.loc[i, 'MainResidue']=liste_loc[0].split(" ")[-1]
    if len(liste_loc)>1:   
        smaller.loc[i, 'SecondFeat']=" ".join(liste_loc[1].split(" ")[:-1])
        smaller.loc[i, 'SecondResidue']=liste_loc[1].split(" ")[-1]
    if len(liste_loc)>2:
        print(liste_loc)
        smaller.loc[i, 'ThirdFeat']=" ".join(liste_loc[2].split(" ")[:-1])
        smaller.loc[i, 'ThirdResidue']=liste_loc[2].split(" ")[-1]

In [705]:
liste_loc

['ORF1b 1729', "3'-to-5' exonuclease 206"]

In [706]:
smaller

,syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,Gene,Main_ref_codon,Main_alt_codon,Main_ref_residue,...,ThirdFeat,ThirdResidue,Second_ref_codon,Second_alt_codon,Second_ref_residue,Second_alt_residue,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue
0,0,3,0,0,28312,28310,N 13,Cgc,Ggc,R,...,na,na,na,na,na,na,na,na,na,na
1,0,2,0,0,25563,25561,ORF3a 57,caG,"caA,caT",Q,...,na,na,na,na,na,na,na,na,na,na
2,0,4,0,0,1060,1058,"ORF1a 265,nsp2 85",acC,acT,T,...,na,na,na,na,na,na,na,na,na,na
3,0,3,0,0,10324,10322,"ORF1a 3353,3C-like proteinase 90",aaG,aaT,K,...,na,na,na,na,na,na,na,na,na,na
4,0,3,0,0,11083,11081,"ORF1a 3606,nsp6 37",ttG,ttT,L,...,na,na,na,na,na,na,na,na,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,0,1,0,0,16536,16534,"ORF1b 1022,helicase 100",0,0,S,...,na,na,na,na,na,na,na,na,na,na
122,2,0,0,0,17247,17245,"ORF1b 1259,helicase 337",0,0,R,...,na,na,na,na,na,na,na,na,na,na
123,0,1,0,0,17280,17278,"ORF1b 1270,helicase 348",0,0,V,...,na,na,na,na,na,na,na,na,na,na
124,0,0,0,0,17799,17797,"ORF1b 1443,helicase 521",0,0,V,...,na,na,na,na,na,na,na,na,na,na


In [707]:
workingtable=smaller.drop(labels='Gene', axis=1)

In [708]:
workingtable.SecondFeat.unique()

array(['na', 'nsp2', '3C-like proteinase', 'nsp6', 'RdRp', 'nsp3', 'nsp4',
       'leader protein', 'helicase', "3'-to-5' exonuclease", 'nsp7',
       'nsp8'], dtype=object)

In [709]:
workingtable.MainFeat.unique()

array(['N', 'ORF3a', 'ORF1a', 'ORF1b', 'S', 'ORF7a', 'ORF8', 'M', 'ORF6'],
      dtype=object)

In [710]:
dic={}

In [711]:
for i in workingtable['MainFeat'].unique():
    dic[i]=workingtable[workingtable['MainFeat']==i]

In [712]:
dic['ORF1b']

,syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,Main_ref_codon,Main_alt_codon,Main_ref_residue,Main_alt_residue,...,ThirdFeat,ThirdResidue,Second_ref_codon,Second_alt_codon,Second_ref_residue,Second_alt_residue,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue
5,0,2,0,0,13731,13729,gCt,gTt,A,V,...,na,na,na,na,na,na,na,na,na,na
6,0,2,0,0,14409,14407,cCt,cTt,P,L,...,na,na,na,na,na,na,na,na,na,na
7,0,2,0,0,14787,14785,gCt,gTt,A,V,...,na,na,na,na,na,na,na,na,na,na
31,0,1,0,0,13629,13627,Gac,Tac,D,Y,...,na,na,na,na,na,na,na,na,na,na
32,0,1,0,0,15279,15277,Cac,Tac,H,Y,...,na,na,na,na,na,na,na,na,na,na
62,0,3,0,0,13515,13513,0,0,G,"Y,C",...,na,na,na,na,na,na,na,na,na,na
63,0,1,0,0,17127,17125,0,0,L,"P,F,I",...,na,na,na,na,na,na,na,na,na,na
64,0,1,0,0,18654,18652,Cgc,Tgc,R,C,...,na,na,na,na,na,na,na,na,na,na
65,0,1,0,0,18687,18685,Tgc,Cgc,C,R,...,na,na,na,na,na,na,na,na,na,na
119,1,0,0,0,13947,13945,0,0,P,,...,na,na,na,na,na,na,na,na,na,na


## Import Feature data from NCBI Genbank File

In [713]:
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import SeqIO


In [714]:
recs = [rec for rec in SeqIO.parse("annotations.gb", "genbank")]


Get Features in a Dataframe, and add ORF1b that is not in the GenBank File

In [788]:
features_list=[{'gene': 'ORF1b polyprotein', 'start': 13467, 'end': 21555}]
for rec in recs:
    feats = [feat for feat in rec.features if feat.type == "mat_peptide" or feat.type == "CDS"]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        res={'gene':product,'start':start,'end':end}
        features_list.append(res)
    feats = [feat for feat in rec.features if feat.type == "CDS"]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        res={'gene':product,'start':start,'end':end}
        features_list.append(res)

In [797]:
features_list=[{'gene': 'ORF1b polyprotein', 'start': 13467, 'end': 21555}]
for rec in recs:
    feats = [print(feat.location) for feat in rec.features if feat.type == "CDS"]

join{[265:13468](+), [13467:21555](+)}
[265:13483](+)
[21562:25384](+)
[25392:26220](+)
[26244:26472](+)
[26522:27191](+)
[27201:27387](+)
[27393:27759](+)
[27755:27887](+)
[27893:28259](+)
[28273:29533](+)
[29557:29674](+)


In [831]:
features_list=[{'gene': 'ORF1b polyprotein', 'start': 13467, 'end': 21555},{'gene': 'RdRp1', 'start': 13441, 'end': 13468},{'gene': 'RdRp2', 'start': 13467, 'end': 16236}]
for rec in recs:
    feats = [feat for feat in rec.features if feat.type == "mat_peptide" or feat.type == "CDS"]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        res={'gene':product,'start':start,'end':end}
        features_list.append(res)
    feats = [feat for feat in rec.features if feat.type == "CDS"]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        res={'gene':product,'start':start,'end':end}
        features_list.append(res)


In [832]:
features_mat=pd.DataFrame(features_list)

In [833]:
features_mat

,gene,start,end
0,ORF1b polyprotein,13467,21555
1,RdRp1,13441,13468
2,RdRp2,13467,16236
3,ORF1ab polyprotein,265,21555
4,leader protein,265,805
5,nsp2,805,2719
6,nsp3,2719,8554
7,nsp4,8554,10054
8,3C-like proteinase,10054,10972
9,nsp6,10972,11842


In [834]:
features_mat=features_mat.drop_duplicates().sort_values(by=['start']).reset_index()[['gene','start','end']]

In [835]:
features_mat['short']='na'
for i,row in features_mat.iterrows():
    features_mat.loc[i, 'short']=row['gene'].split(' ')[0]



In [836]:
features_mat

,gene,start,end,short
0,ORF1a polyprotein,265,13483,ORF1a
1,ORF1ab polyprotein,265,21555,ORF1ab
2,leader protein,265,805,leader
3,nsp2,805,2719,nsp2
4,nsp3,2719,8554,nsp3
5,nsp4,8554,10054,nsp4
6,3C-like proteinase,10054,10972,3C-like
7,nsp6,10972,11842,nsp6
8,nsp7,11842,12091,nsp7
9,nsp8,12091,12685,nsp8


Removing ORF1ab From the table

In [837]:
features_mat=features_mat.drop(labels=[1,13,14], axis=0)

In [847]:
features_mat['Length']=(features_mat['end']-features_mat['start'])/3
features_mat['Length']=features_mat['Length'].astype('int64')

In [848]:
features_mat

,gene,start,end,short,Length
0,ORF1a polyprotein,265,13483,ORF1a,4406
2,leader protein,265,805,leader,180
3,nsp2,805,2719,nsp2,638
4,nsp3,2719,8554,nsp3,1945
5,nsp4,8554,10054,nsp4,500
6,3C-like proteinase,10054,10972,3C-like,306
7,nsp6,10972,11842,nsp6,290
8,nsp7,11842,12091,nsp7,83
9,nsp8,12091,12685,nsp8,198
10,nsp9,12685,13024,nsp9,113


Get Genome sequence to get codon informations

In [849]:
!head sequence.fasta


>NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAA
CGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAAC
TAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTG
TTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTC
CCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTAC
GTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGG
CTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGAT
GCTCGAACTGCACCTCATGGTCATGTTATGGTTGAGCTGGTAGCAGAACTCGAAGGCATTCAGTACGGTC
GTAGTGGTGAGACACTTGGTGTCCTTGTCCCTCATGTGGGCGAAATACCAGTGGCTTACCGCAAGGTTCT


In [850]:
with open('sequence.fasta') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 

In [851]:
gen_seq=''.join(content[1:])

In [852]:
len(gen_seq)

29903

In [853]:
workingtable['MainFeat'].unique()

array(['N', 'ORF3a', 'ORF1a', 'ORF1b', 'S', 'ORF7a', 'ORF8', 'M'],
      dtype=object)

In [854]:
list(workingtable['SecondFeat'].unique())

['na',
 'nsp2',
 '3C-like proteinase',
 'nsp6',
 'RdRp',
 'nsp3',
 'nsp4',
 'leader protein',
 'helicase',
 "3'-to-5' exonuclease"]

## Add Feature information to variant tables

In [855]:
workingtable.head(5)

,syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,Main_ref_codon,Main_alt_codon,Main_ref_residue,Main_alt_residue,...,Second_alt_codon,Second_ref_residue,Second_alt_residue,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,POS,AF,type
0,0,3,na,0,28312,28310,Cgc,Ggc,R,G,...,na,na,na,na,na,na,na,28310,0,S
1,0,2,na,0,25563,25561,caG,"caA,caT",Q,"Q,H",...,na,na,na,na,na,na,na,25561,0,S
2,0,4,na,0,1060,1058,acC,acT,T,T,...,na,na,na,na,na,na,na,1058,0,S
3,0,3,na,0,10324,10322,aaG,aaT,K,N,...,na,na,na,na,na,na,na,10322,0,S
4,0,3,na,0,11083,11081,ttG,ttT,L,F,...,na,na,na,na,na,na,na,11081,0,S


In [856]:
synonymous

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,IMPACT,...,Second_ref_residue,Second_alt_residue,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,Genomic_coord_end,Genomic_coord_start
2,SRR11397714,NC_045512,25563,G,T,119,1.000000,0,"0,0,56,63",MODERATE,...,na,na,na,na,na,na,na,na,25563,25561
3,SRR11494669,NC_045512,25563,G,T,160,1.000000,0,"0,0,74,86",MODERATE,...,na,na,na,na,na,na,na,na,25563,25561
4,SRR11494527,NC_045512,25563,G,T,137,1.000000,0,"0,0,57,80",MODERATE,...,na,na,na,na,na,na,na,na,25563,25561
5,SRR11494644,NC_045512,25563,G,T,203,1.000000,0,"0,0,111,92",MODERATE,...,na,na,na,na,na,na,na,na,25563,25561
6,SRR11494701,NC_045512,25563,G,T,149,1.000000,0,"0,0,69,80",MODERATE,...,na,na,na,na,na,na,na,na,25563,25561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4139,SRR11410542,NC_045512,11751,T,C,139,0.050360,1,"35,97,1,6",MODERATE,...,L,P,na,na,na,na,na,na,11752,11750
4141,SRR11494532,NC_045512,19159,T,C,159,0.050314,1,"58,93,4,4",MODERATE,...,S,P,na,na,na,na,na,na,19161,19159
4142,SRR11494664,NC_045512,24131,G,T,179,0.050279,1,"81,89,5,4",MODERATE,...,na,na,na,na,na,na,na,na,24133,24131
4143,SRR11494664,NC_045512,23903,G,T,179,0.050279,0,"79,91,4,5",MODERATE,...,na,na,na,na,na,na,na,na,23905,23903


In [857]:
synonymous['syn']=0
synonymous['non_syn']=1
synonymous['MainResidue']='na'
synonymous['MainFeat']='na'
synonymous['Main_ref_codon']='na'
synonymous['Main_alt_codon']='na'
synonymous['Main_ref_residue']='na'
synonymous['Main_alt_residue']='na'
synonymous['SecondFeat']='na'
synonymous['SecondResidue']='na'
synonymous['Second_ref_codon']='na'
synonymous['Second_alt_codon']='na'
synonymous['Second_ref_residue']='na'
synonymous['Second_alt_residue']='na'
synonymous['ThirdResidue']='na'
synonymous['ThirdFeat']='na'
synonymous['Third_ref_codon']='na'
synonymous['Third_alt_codon']='na'
synonymous['Third_ref_residue']='na'
synonymous['Third_alt_residue']='na'
synonymous['Genomic_coord_end']=0
synonymous['Genomic_coord_start']=0
for i,row in synonymous.iterrows():
    #synonymous.loc[i, 'ref_codon']=row['CODON'].split('/')[0]
    #synonymous.loc[i, 'alt_codon']=row['CODON'].split('/')[1]
    #synonymous.loc[i, 'ref_residue']=Bio.Seq._translate_str(row['CODON'].split('/')[0], table)
    #synonymous.loc[i, 'alt_residue']=','.join([Bio.Seq._translate_str(row['CODON'].split('/')[1], table) for i in row['CODON'].split(',')])
    features=list(features_mat[(features_mat.start<row['POS']) & (features_mat.end>row['POS'])]['gene'])
    
    synonymous.loc[i, 'MainFeat']=features[0]
    startmain=int(features_mat[features_mat['gene']==features[0]].start)
    if features[0]=='RdRp2':
        synonymous.loc[i, 'MainResidue']=math.ceil((row['POS']-(startmain))/3)+9
    else:
        synonymous.loc[i, 'MainResidue']=math.ceil((row['POS']-(startmain))/3)
    if (row['POS']-startmain)%3==0:
        startposmain=row['POS']-2
    else :
        startposmain=row['POS']+1-((row['POS']-startmain)%3)
    synonymous.loc[i, 'Genomic_coord_start']=startposmain
    synonymous.loc[i, 'Genomic_coord_end']=startposmain+2
    maincod=gen_seq[(startposmain-1):(startposmain-1)+3]
    index=(row['POS']-startmain)%3
    mainaltcod=maincod[:(row['POS']-startposmain)] + row['ALT'] +  maincod[(row['POS']-startposmain)+1:]
    synonymous.loc[i, 'Main_ref_codon']=maincod
    synonymous.loc[i, 'Main_alt_codon']=mainaltcod
    synonymous.loc[i, 'Main_ref_residue']=Bio.Seq._translate_str(maincod, table)
    synonymous.loc[i, 'Main_alt_residue']=Bio.Seq._translate_str(mainaltcod, table) 
    if len(features)>=2 :
        synonymous.loc[i, 'SecondFeat']=features[1]
        startsecond=int(features_mat[features_mat['gene']==features[1]].start)
        if features[1]=='RdRp2':
            synonymous.loc[i, 'SecondResidue']=math.ceil((row['POS']-(startsecond))/3)+9
        else:
            synonymous.loc[i, 'SecondResidue']=math.ceil((row['POS']-(startsecond))/3)
        if (row['POS']-startsecond)%3==0:
            startpossec=row['POS']-2
        else :
            startpossec=row['POS']+1-((row['POS']-startsecond)%3)
        synonymous.loc[i, 'Genomic_coord_start']=startpossec
        synonymous.loc[i, 'Genomic_coord_end']=startpossec+2
        seccod=gen_seq[(startpossec-1):(startpossec-1)+3]
        index=(row['POS']-startsecond)%3
        secaltcod=seccod[:(row['POS']-startpossec)] + row['ALT'] +  seccod[(row['POS']-startpossec)+1:]
        synonymous.loc[i, 'Second_ref_codon']=seccod
        synonymous.loc[i, 'Second_alt_codon']=secaltcod
        synonymous.loc[i, 'Second_ref_residue']=Bio.Seq._translate_str(seccod, table)
        synonymous.loc[i, 'Second_alt_residue']=Bio.Seq._translate_str(secaltcod, table) 
    if len(features)>2:
        print(features)
        synonymous.loc[i, 'ThirdFeat']=features[2]
        startthird=int(features_mat[features_mat['gene']==features[2]].start)
        if features[2]=='RdRp2':
            synonymous.loc[i, 'ThirdResidue']=math.ceil((row['POS']-(startthird))/3)+9
        else:
            synonymous.loc[i, 'ThirdResidue']=math.ceil((row['POS']-(startthird))/3)
        if (row['POS']-startthird)%3==0:
            startposthird=row['POS']-2
        else :
            startposthird=row['POS']+1-((row['POS']-startthird)%3)
        synonymous.loc[i, 'Genomic_coord_start']=startposthird
        synonymous.loc[i, 'Genomic_coord_end']=startposthird+2
        thirdcod=gen_seq[(startposthird-1):(startposthird-1)+3]
        index=(row['POS']-startthird)%3
        thirdaltcod=thirdcod[:(row['POS']-startposthird)] + row['ALT'] +  thirdcod[(row['POS']-startposthird)+1:]
        synonymous.loc[i, 'Third_ref_codon']=thirdcod
        synonymous.loc[i, 'Third_alt_codon']=thirdaltcod
        synonymous.loc[i, 'Third_ref_residue']=Bio.Seq._translate_str(thirdcod, table)
        synonymous.loc[i, 'Third_alt_residue']=Bio.Seq._translate_str(thirdaltcod, table) 
        print(thirdcod)
        


['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
CGG
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
CGG


In [864]:
synonymous.MainFeat.unique()

array(['ORF3a protein', 'surface glycoprotein', 'ORF1b polyprotein',
       'RdRp2', 'ORF1a polyprotein', 'nucleocapsid phosphoprotein',
       'ORF7a protein', 'ORF8 protein', 'envelope protein',
       'ORF10 protein', 'membrane glycoprotein', 'ORF6 protein', 'ORF7b'],
      dtype=object)

In [867]:
synonymous[synonymous['MainFeat']=='RdRp2']

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,IMPACT,...,Second_ref_residue,Second_alt_residue,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,Genomic_coord_end,Genomic_coord_start
27,SRR11494661,NC_045512,15205,A,G,129,1.000000,0,"0,0,58,71",MODERATE,...,I,V,na,na,na,na,na,na,15207,15205
34,SRR11494585,NC_045512,14408,C,T,241,1.000000,0,"0,0,121,120",MODERATE,...,P,L,na,na,na,na,na,na,14409,14407
35,SRR11494627,NC_045512,14408,C,T,211,1.000000,0,"0,0,106,105",MODERATE,...,P,L,na,na,na,na,na,na,14409,14407
36,SRR11494631,NC_045512,14408,C,T,235,1.000000,0,"0,0,116,119",MODERATE,...,P,L,na,na,na,na,na,na,14409,14407
37,SRR11494640,NC_045512,14408,C,T,219,1.000000,0,"0,0,107,112",MODERATE,...,P,L,na,na,na,na,na,na,14409,14407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4094,SRR11494646,NC_045512,15202,G,T,156,0.051282,1,"67,80,3,5",MODERATE,...,V,L,na,na,na,na,na,na,15204,15202
4095,SRR11494664,NC_045512,14294,G,T,195,0.051282,13,"68,117,7,3",MODERATE,...,R,L,na,na,na,na,na,na,14295,14293
4101,SRR11494646,NC_045512,14197,G,T,176,0.051136,1,"86,81,4,5",MODERATE,...,A,S,na,na,na,na,na,na,14199,14197
4105,SRR11494657,NC_045512,15193,G,T,137,0.051095,0,"64,66,3,4",MODERATE,...,A,S,na,na,na,na,na,na,15195,15193


### Select relevant collumns

In [868]:
work2=synonymous[['POS', 'REF', 'ALT', 'AF','type', 'syn',
       'non_syn','MainResidue', 'MainFeat', 'Main_ref_codon',
       'Main_alt_codon', 'Main_ref_residue', 'Main_alt_residue', 'SecondFeat',
       'SecondResidue', 'Second_ref_codon', 'Second_alt_codon',
       'Second_ref_residue', 'Second_alt_residue', 'ThirdResidue', 'ThirdFeat',
       'Third_ref_codon', 'Third_alt_codon', 'Third_ref_residue',
       'Third_alt_residue', 'Genomic_coord_end', 'Genomic_coord_start','NbSample','MeanAF']]

In [869]:
work2

,POS,REF,ALT,AF,type,syn,non_syn,MainResidue,MainFeat,Main_ref_codon,...,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,Genomic_coord_end,Genomic_coord_start,NbSample,MeanAF
2,25563,G,T,1.000000,S,0,1,57,ORF3a protein,CAG,...,na,na,na,na,na,na,25563,25561,90,0.965740
3,25563,G,T,1.000000,S,0,1,57,ORF3a protein,CAG,...,na,na,na,na,na,na,25563,25561,90,0.965740
4,25563,G,T,1.000000,S,0,1,57,ORF3a protein,CAG,...,na,na,na,na,na,na,25563,25561,90,0.965740
5,25563,G,T,1.000000,S,0,1,57,ORF3a protein,CAG,...,na,na,na,na,na,na,25563,25561,90,0.965740
6,25563,G,T,1.000000,S,0,1,57,ORF3a protein,CAG,...,na,na,na,na,na,na,25563,25561,90,0.965740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4139,11751,T,C,0.050360,S,0,1,3829,ORF1a polyprotein,CTC,...,na,na,na,na,na,na,11752,11750,1,0.050360
4141,19159,T,C,0.050314,S,0,1,1898,ORF1b polyprotein,TCT,...,na,na,na,na,na,na,19161,19159,1,0.050314
4142,24131,G,T,0.050279,S,0,1,857,surface glycoprotein,GGC,...,na,na,na,na,na,na,24133,24131,1,0.050279
4143,23903,G,T,0.050279,S,0,1,781,surface glycoprotein,GTT,...,na,na,na,na,na,na,23905,23903,1,0.050279


In [870]:
work2=work2.drop_duplicates()

In [871]:
workingtable['POS']=workingtable['Genomic_coord_start']
workingtable['REF']='na'
workingtable['AF']='0'
workingtable['type']='NS'

In [872]:
for i,row in workingtable.iterrows():
    if int(row.non_syn)>0:
        workingtable.loc[i, 'type']='S'

Select only non synonymous mutations

In [873]:
workingtable=workingtable[workingtable.type=='S']

In [874]:
work2

,POS,REF,ALT,AF,type,syn,non_syn,MainResidue,MainFeat,Main_ref_codon,...,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,Genomic_coord_end,Genomic_coord_start,NbSample,MeanAF
2,25563,G,T,1.000000,S,0,1,57,ORF3a protein,CAG,...,na,na,na,na,na,na,25563,25561,90,0.965740
13,23403,A,G,1.000000,S,0,1,614,surface glycoprotein,GAT,...,na,na,na,na,na,na,23404,23402,195,0.960663
18,21710,T,C,1.000000,S,0,1,50,surface glycoprotein,TCA,...,na,na,na,na,na,na,21712,21710,1,1.000000
19,20578,G,T,1.000000,S,0,1,2371,ORF1b polyprotein,GTG,...,na,na,na,na,na,na,20580,20578,6,0.834473
22,19684,G,T,1.000000,S,0,1,2073,ORF1b polyprotein,GTA,...,na,na,na,na,na,na,19686,19684,23,0.969106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4139,11751,T,C,0.050360,S,0,1,3829,ORF1a polyprotein,CTC,...,na,na,na,na,na,na,11752,11750,1,0.050360
4141,19159,T,C,0.050314,S,0,1,1898,ORF1b polyprotein,TCT,...,na,na,na,na,na,na,19161,19159,1,0.050314
4142,24131,G,T,0.050279,S,0,1,857,surface glycoprotein,GGC,...,na,na,na,na,na,na,24133,24131,1,0.050279
4143,23903,G,T,0.050279,S,0,1,781,surface glycoprotein,GTT,...,na,na,na,na,na,na,23905,23903,1,0.050279


In [875]:
workingtable

,syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,Main_ref_codon,Main_alt_codon,Main_ref_residue,Main_alt_residue,...,Second_alt_codon,Second_ref_residue,Second_alt_residue,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,POS,AF,type
0,0,3,na,0,28312,28310,Cgc,Ggc,R,G,...,na,na,na,na,na,na,na,28310,0,S
1,0,2,na,0,25563,25561,caG,"caA,caT",Q,"Q,H",...,na,na,na,na,na,na,na,25561,0,S
2,0,4,na,0,1060,1058,acC,acT,T,T,...,na,na,na,na,na,na,na,1058,0,S
3,0,3,na,0,10324,10322,aaG,aaT,K,N,...,na,na,na,na,na,na,na,10322,0,S
4,0,3,na,0,11083,11081,ttG,ttT,L,F,...,na,na,na,na,na,na,na,11081,0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,0,1,na,0,6541,6539,0,0,H,Y,...,na,na,na,na,na,na,na,6539,0,S
116,0,1,na,0,9112,9110,0,0,R,H,...,na,na,na,na,na,na,na,9110,0,S
121,0,1,na,0,16536,16534,0,0,S,"I,G",...,na,na,na,na,na,na,na,16534,0,S
123,0,1,na,0,17280,17278,0,0,V,L,...,na,na,na,na,na,na,na,17278,0,S


Concatenate Variant Tables

In [876]:
total=pd.concat([work2,workingtable])

In [877]:
total.POS=total.POS.astype('int64')

In [878]:
total=total.sort_values(by=['POS']).reset_index()

In [879]:
total.head(20)


,index,POS,REF,ALT,AF,type,syn,non_syn,MainResidue,MainFeat,...,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,Genomic_coord_end,Genomic_coord_start,NbSample,MeanAF
0,3718,269,G,T,0.06087,S,0,1,2,ORF1a polyprotein,...,na,na,na,na,na,na,271,269,1,0.060870
1,3471,275,C,T,0.071429,S,0,1,4,ORF1a polyprotein,...,na,na,na,na,na,na,277,275,1,0.071429
2,3306,285,G,T,0.079545,S,0,1,7,ORF1a polyprotein,...,na,na,na,na,na,na,286,284,4,0.137573
3,3020,285,G,T,0.102703,S,0,1,7,ORF1a polyprotein,...,na,na,na,na,na,na,286,284,4,0.137573
4,2482,285,G,T,0.29798,S,0,1,7,ORF1a polyprotein,...,na,na,na,na,na,na,286,284,4,0.137573
5,3498,285,G,T,0.070064,S,0,1,7,ORF1a polyprotein,...,na,na,na,na,na,na,286,284,4,0.137573
6,3386,308,C,A,0.075269,S,0,1,15,ORF1a polyprotein,...,na,na,na,na,na,na,310,308,2,0.067854
7,3725,308,C,A,0.06044,S,0,1,15,ORF1a polyprotein,...,na,na,na,na,na,na,310,308,2,0.067854
8,3888,312,T,A,0.055866,S,0,1,16,ORF1a polyprotein,...,na,na,na,na,na,na,313,311,1,0.055866
9,3480,350,C,T,0.070539,S,0,1,29,ORF1a polyprotein,...,na,na,na,na,na,na,352,350,1,0.070539


In [880]:
total['MainFeat']=[re.sub(r"membrane glycoprotein","M",j) for j in [re.sub(r" (poly)?protein",'',i) for i in total.MainFeat]]
total['MainFeat']=[re.sub(r"RNA-dependent RNA polymerase","RdRp",j) for j in [re.sub(r"nucleocapsid phosphoprotein",'N',i) for i in total.MainFeat]]
total['MainFeat']=[re.sub(r"envelope","E",j) for j in [re.sub(r"surface",'S',i) for i in total.MainFeat]]
total['MainFeat']=[re.sub(r"S glycoprotein","S",j) for j in [re.sub(r"membrane",'M',i) for i in total.MainFeat]]

In [881]:
total['MainFeat']

0       ORF1a
1       ORF1a
2       ORF1a
3       ORF1a
4       ORF1a
        ...  
2715    ORF10
2716    ORF10
2717    ORF10
2718    ORF10
2719    ORF10
Name: MainFeat, Length: 2720, dtype: object

In [882]:
total['SecondFeat']=[re.sub(r"membrane glycoprotein","M",j) for j in [re.sub(r" (poly)?protein",'',i) for i in total.SecondFeat]]
total['SecondFeat']=[re.sub(r"RNA-dependent RNA polymerase","RdRp",j) for j in [re.sub(r"nucleocapsid phosphoprotein",'N',i) for i in total.SecondFeat]]
total['SecondFeat']=[re.sub(r"envelope","E",j) for j in [re.sub(r"surface",'S',i) for i in total.SecondFeat]]
total['SecondFeat']=[re.sub(r"membrane",'M',i) for i in total.SecondFeat]

In [883]:
list(set(total.SecondFeat.unique())|set(total.MainFeat.unique()))

['RdRp',
 'nsp2',
 'ORF10',
 'helicase',
 'nsp8',
 'nsp9',
 'ORF7b',
 'nsp10',
 'RdRp1',
 'N',
 'ORF1b',
 'RdRp2',
 'nsp7',
 "3'-to-5' exonuclease",
 'leader',
 'nsp6',
 "2'-O-ribose methyltransferase",
 'ORF8',
 'nsp3',
 'endoRNAse',
 'ORF6',
 'ORF1a',
 'ORF3a',
 'E',
 'na',
 'M',
 'nsp4',
 'S',
 'ORF7a',
 '3C-likease']

In [884]:
total['ThirdFeat']=[re.sub(r"membrane glycoprotein","M",j) for j in [re.sub(r" (poly)?protein",'',i) for i in total.ThirdFeat]]
total['ThirdFeat']=[re.sub(r"RNA-dependent RNA polymerase","RdRp",j) for j in [re.sub(r"nucleocapsid phosphoprotein",'N',i) for i in total.ThirdFeat]]
total['ThirdFeat']=[re.sub(r"envelope","E",j) for j in [re.sub(r"surface",'S',i) for i in total.ThirdFeat]]
total['ThirdFeat']=[re.sub(r"membrane",'M',i) for i in total.ThirdFeat]

In [885]:
list(set(total.SecondFeat.unique())|set(total.MainFeat.unique())|set(total.ThirdFeat.unique()))

['RdRp',
 'nsp2',
 'ORF10',
 'helicase',
 'nsp8',
 'nsp9',
 'ORF7b',
 'nsp10',
 'RdRp1',
 'N',
 'ORF1b',
 'RdRp2',
 'nsp7',
 "3'-to-5' exonuclease",
 'leader',
 'nsp6',
 "2'-O-ribose methyltransferase",
 'ORF8',
 'nsp3',
 'endoRNAse',
 'ORF6',
 'ORF1a',
 'ORF3a',
 'E',
 'na',
 'M',
 'nsp4',
 'S',
 'ORF7a',
 '3C-likease']

In [886]:
total.columns

Index(['index', 'POS', 'REF', 'ALT', 'AF', 'type', 'syn', 'non_syn',
       'MainResidue', 'MainFeat', 'Main_ref_codon', 'Main_alt_codon',
       'Main_ref_residue', 'Main_alt_residue', 'SecondFeat', 'SecondResidue',
       'Second_ref_codon', 'Second_alt_codon', 'Second_ref_residue',
       'Second_alt_residue', 'ThirdResidue', 'ThirdFeat', 'Third_ref_codon',
       'Third_alt_codon', 'Third_ref_residue', 'Third_alt_residue',
       'Genomic_coord_end', 'Genomic_coord_start', 'NbSample', 'MeanAF'],
      dtype='object')

In [887]:
#total=total[['POS', 'REF', 'ALT', 'AF', 'type', 'syn', 'non_syn', 'ref_codon', 'alt_codon', 'ref_residue', 'alt_residue', 'Genomic_coord_start', 'Genomic_coord_end', 'cleanMain', 'MainResidue','cleanSecond', 'SecondResidue','cleanThird','ThirdResidue','NbSample','MeanAF']]


Create Tables for each Features

In [888]:
total

,index,POS,REF,ALT,AF,type,syn,non_syn,MainResidue,MainFeat,...,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,Genomic_coord_end,Genomic_coord_start,NbSample,MeanAF
0,3718,269,G,T,0.06087,S,0,1,2,ORF1a,...,na,na,na,na,na,na,271,269,1,0.060870
1,3471,275,C,T,0.071429,S,0,1,4,ORF1a,...,na,na,na,na,na,na,277,275,1,0.071429
2,3306,285,G,T,0.079545,S,0,1,7,ORF1a,...,na,na,na,na,na,na,286,284,4,0.137573
3,3020,285,G,T,0.102703,S,0,1,7,ORF1a,...,na,na,na,na,na,na,286,284,4,0.137573
4,2482,285,G,T,0.29798,S,0,1,7,ORF1a,...,na,na,na,na,na,na,286,284,4,0.137573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2715,3863,29612,T,C,0.056738,S,0,1,19,ORF10,...,na,na,na,na,na,na,29614,29612,1,0.056738
2716,670,29625,C,T,0.981366,S,0,1,23,ORF10,...,na,na,na,na,na,na,29626,29624,1,0.981366
2717,3972,29642,C,T,0.054217,S,0,1,29,ORF10,...,na,na,na,na,na,na,29644,29642,3,0.316471
2718,2275,29642,C,T,0.796296,S,0,1,29,ORF10,...,na,na,na,na,na,na,29644,29642,3,0.316471


In [889]:
dicfeatures={}
for i in list(set(total.SecondFeat.unique())|set(total.MainFeat.unique())|set(total.ThirdFeat.unique())) :
    dicfeatures[i]=total[(total.MainFeat==i) | (total.SecondFeat==i)| (total.ThirdFeat==i)]
    dicfeatures[i]=dicfeatures[i].drop(labels=['syn', 'non_syn','index'], axis=1)
    if np.all([[dicfeatures[i].ThirdFeat.unique()==['na']]]):
        dicfeatures[i]=dicfeatures[i].drop(labels=['ThirdResidue', 'ThirdFeat', 'Third_ref_codon','Third_alt_codon', 'Third_ref_residue', 'Third_alt_residue'], axis=1)
        if np.all([[dicfeatures[i].SecondFeat.unique()==['na']]]):
            dicfeatures[i]=dicfeatures[i].drop(labels=['SecondFeat', 'SecondResidue','Second_ref_codon', 'Second_alt_codon', 'Second_ref_residue','Second_alt_residue'], axis=1)

In [891]:
dicfeatures['RdRp2'][['SecondFeat', 'SecondResidue','Second_ref_codon', 'Second_alt_codon', 'Second_ref_residue','Second_alt_residue']].head(30)

,SecondFeat,SecondResidue,Second_ref_codon,Second_alt_codon,Second_ref_residue,Second_alt_residue
998,RdRp2,10,CGG,CTG,R,L
999,ORF1b,13,TGC,GGC,C,G
1000,ORF1b,14,GGC,TGC,G,C
1002,ORF1b,35,GGT,GTT,G,V
1003,ORF1b,35,GGT,GTT,G,V
1004,ORF1b,35,GGT,GTT,G,V
1005,ORF1b,35,GGT,GTT,G,V
1006,ORF1b,35,GGT,GTT,G,V
1007,ORF1b,35,GGT,GTT,G,V
1008,ORF1b,46,CGC,AGC,R,S


In [893]:
dicfeatures['RdRp']=pd.concat([dicfeatures['RdRp'],dicfeatures['RdRp2']])
dicfeatures['RdRp']=pd.concat([dicfeatures['RdRp'],dicfeatures['RdRp1']])

In [894]:
dicfeatures['RdRp']

,POS,REF,ALT,AF,type,MainResidue,MainFeat,Main_ref_codon,Main_alt_codon,Main_ref_residue,...,Genomic_coord_end,Genomic_coord_start,NbSample,MeanAF,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue
1001,13513,na,0,0,S,15,ORF1b,0,0,G,...,13515,13513,3,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1012,13627,na,0,0,S,53,ORF1b,Gac,Tac,D,...,13629,13627,7,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1020,13729,na,0,0,S,87,ORF1b,gCt,gTt,A,...,13731,13729,4,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1065,14407,na,0,0,S,313,ORF1b,cCt,cTt,P,...,14409,14407,8,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1237,14785,na,0,0,S,439,ORF1b,gCt,gTt,A,...,14787,14785,3,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,16207,G,T,0.051429,S,923,RdRp2,GCT,TCT,A,...,16209,16207,1,0.051429,na,na,na,na,na,na
1301,16222,C,T,0.994012,S,928,RdRp2,CAT,TAT,H,...,16224,16222,2,0.981855,na,na,na,na,na,na
1302,16222,C,T,0.969697,S,928,RdRp2,CAT,TAT,H,...,16224,16222,2,0.981855,na,na,na,na,na,na
996,13454,C,T,0.188406,S,4397,ORF1a,CAA,TAA,Q,...,13456,13454,1,0.188406,NaN,NaN,NaN,NaN,NaN,NaN


In [895]:
for i in dicfeatures.keys():
    dicfeatures[i].to_csv(i+'_var.csv',  sep='\t',index=False, header=True)